In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

df_team_names = pd.read_csv('inputs/all_sources_team_names.csv')
team_names = df_team_names['fanfooty_team_name_long'].tolist()
url = "http://www.fanfooty.com.au/aflteams/"
df_player_list = pd.DataFrame()

for team in team_names:
    row_list = []
    column_headers = []
    res = requests.get("{}{}/".format(url, team))
    soup = BeautifulSoup(res.text, features='lxml')
    table = soup.select("#content-area > table:nth-child(10)")
    table = table[0]
    th = table.findAll(lambda tag: tag.name=='th')
    for header in th:
        column_headers.append(header.text)
    rows = table.findAll(lambda tag: tag.name=='tr')
    for tr in rows[1:]:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        row_list.append(row)
    df = pd.DataFrame(row_list, columns=column_headers)
    df['team'] = team
    df_player_list = pd.concat([df_player_list, df])
    
df_player_list['short_team'] = df_player_list['team'].map(dict(df_team_names[['fanfooty_team_name_long', 'footywire_team_name']].values))
df_player_list